In [1]:
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2
import pandas as pd
import pickle as pkl
import warnings
warnings.simplefilter(action='ignore')
import numpy as np
from notebook_utils import *
from parameter_estimation import load_data_util
import numpy as np
import pandas as pd
import snap
import operator
from sklearn import metrics
import scikitplot as skplt
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='serif', size='13') # controls default text sizes
plt.rc('xtick', labelsize='12')
plt.rc('ytick', labelsize='12')
plt.rc('legend', fontsize='13') # legend fontsize
plt.rc('axes', labelsize='12', titlesize='13')  # fontsize of the x and y labels # fontsize of the axes title
# plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


# Extract tweets features 

In [75]:
#!/usr/bin/env python
# encoding: utf-8
"""A script for downloading (crawling) tweets by their IDs."""

import tweepy
import pickle as pkl

consumerkey = 'zg5ZoWps3wHjfAwjmR5mQHjKW'
consumersecret = 'dsCwQ2Da7ajUTj4IWHTEUY0owJ0GxZOiGVM9Lr0529RqEiRk31'
accesstoken = '936005824111026176-WfU8grlXW2ogKis6vmB1rC6dcAquaCO'
accesssecret = 'nJZBE8h2Y8aloEZnJ4Fuwq3MFJvf6E4BVRx41lwbXqpyc'


if __name__ == '__main__':
    auth = tweepy.OAuthHandler(consumerkey, consumersecret)
    auth.set_access_token(accesstoken, accesssecret)
    api = tweepy.API(auth)
    
    # in_path = './untitled.txt'
    # out_path = './testoutput.pkl'
    
    in_path = '../output/all_kwon/kwon/infl_uids_tweetids.txt'
    out_path = '../output/all_kwon/kwon/output_status_list_infl_uids_tweetids.pkl'
    step = 90
    
    all_tweets = []
    # tweet_ids = []
    tweet_ids = list(np.loadtxt(in_path, dtype=np.int64)[:, 1])
#     f = open(in_path, 'r') 
#     for line in f.readlines():
#         id = int(line.strip().split()[0])
#         tweet_ids.append(id)
        
    for i in range(0, len(tweet_ids), step):
        tweets = api.statuses_lookup(tweet_ids[i:i+step])
        all_tweets += tweets
        # break
        print('done till', i, len(tweets))
        
    pkl.dump(all_tweets, open(out_path, 'wb'))
        
    print('finished')

done till 0 67
done till 90 64
done till 180 17
finished


In [4]:
tweet_ids = list(np.loadtxt('../output/all_kwon/kwon/infl_uids_tweetids.txt', dtype=np.int64)[:, 1])
uids = list(np.loadtxt('../output/all_kwon/kwon/infl_uids_tweetids.txt', dtype=np.int64)[:, 0])
mapp_twid_uid = dict(zip(tweet_ids, uids)) 

all_tweets = pkl.load(open('../output/all_kwon/kwon/output_status_list_infl_uids_tweetids.pkl', 'rb'))
dict_ufeats = {}
for status in all_tweets:
    s = status._json
    twid = s['id']
    uid = mapp_twid_uid[twid]
    # pprint(s)
    uobj = s['user']
    # uid = uobj['id']  # not anonymized, I have anonymized ones in KWON dataset
    desc = uobj['description']
    name = uobj['screen_name']
    veri = uobj['verified']
    tup = (name, desc, veri)
    dict_ufeats[uid] = tup
    # print(uid)
    # break
    # dict_texts[s['id']] = s['text']
print(len(all_tweets), len(dict_ufeats), next(iter(dict_ufeats.values())))

148 117 ('Scooopy', 'http://t.co/4ReKy9iiDb - Scoops about Technology & Gadgets', False)


In [61]:
# extract tweet ids from anonymized user ids

cascades_file = '../data/kwon/cascades.txt'
cascade_names = '../data/kwon/cascade_names.txt'
fil_dir = 'kwon/Tweets/'

train_cascades = read_cascades_file(cascades_file)
casnames = np.loadtxt(cascade_names, dtype=np.str)

def mapping_uidx_to_cid(train_cascades):
    dict_cids = {}
    for cid, cas in enumerate(train_cascades):
        users = cas[:, 0]
        for u in users: dict_cids[int(u)]=cid
    return dict_cids

def get_tweetid_for_uid_from_eventfile(fil_dir, name, uid):
    file_path = fil_dir + name
    df = pd.read_csv(file_path, sep='\t', header=None, usecols=range(4))
    subdf = df[df[0].isin([uid])]
    tweet_id = subdf.iloc[0, 1]
    return tweet_id

def get_list_infl_users():
    # kwon
    fake_infl = [39357, 41645179, 51828921, 1688, 41625921, 904660, 55640892, 45357704, 117511, 31745148, 49130864, 46283660, 31441663, 46161420, 54595164, 39025741, 16983873, 10045, 51489852, 53382638, 19771216, 18987, 85853, 39003758, 26101679, 54795103, 31749799, 40531228, 11402038, 31665361, 2187715, 54888609, 55762225, 12919420, 26206026, 15191995, 44083204, 9866568, 892, 33019416, 4270645, 36020039, 47509244, 49614, 51093837, 45924090, 599, 9581, 34949244, 30715157, 52174399, 34664714, 40179, 45241472, 600250, 26869787, 33498358, 88703, 45887855, 46849, 34753016, 12186498, 56001060, 81502, 10179069, 10054, 25980076, 37733084, 28768064, 51631070, 29708909, 23813323, 46643782, 56009279, 46060230, 2447156, 40134854, 55922657, 17236288, 39758253, 73058, 14808579, 36503735, 43569984, 2267067, 32691858, 41792135, 34882960, 25900052, 47386549, 46730408, 12944483, 9456, 34949531, 10278418, 29030843, 45708734, 15035771, 47419769, 1761233]
    true_infl = [41645179, 51828921, 53471447, 46161420, 54888609, 31745148, 31441663, 117511, 904660, 18987, 15446395, 44282988, 53382638, 51489852, 48439353, 1709, 85853, 26206026, 25532591, 54425344, 45062504, 17512677, 39003758, 31665361, 31749799, 1217410, 2447156, 19771216, 31572449, 15191995, 2187715, 53272550, 48455831, 20155, 16757978, 12919420, 47110694, 44083204, 28555, 45708734, 30889300, 41792135, 1041957, 45887855, 49614, 33603546, 55922657, 13080305, 14808579, 48151623, 892, 73058, 4270645, 61994, 33986646, 46060230, 46849, 47419769, 34891224, 45241472, 48428454, 10278418, 25980076, 48872364, 35478, 31318992, 43981298, 51093837, 55848798, 34711253, 13420494, 9456, 443113, 37484955, 46730408, 5770411, 46931589, 8990, 26101679, 40125, 37733084, 41736970, 53703886, 46295098, 32691858, 37471, 7836260, 27747934, 55778501, 16983873, 26385432, 40864273, 40179, 34882960, 48350639, 46493916, 47386549, 27588149, 31480592, 40074055]
    return fake_infl + true_infl

def find_tweetid_for_userid(list_uids):
    list_tweetids = []
    for uid in list_uids:
        cid = dict_cids[uid]
        name = casnames[int(cid)]
        # find tweet id from cascade file corresponding to this userid.
        tweet_id = get_tweetid_for_uid_from_eventfile(fil_dir, name, uid)
        list_tweetids.append(tweet_id)
        # break
    return list_tweetids

dict_cids = mapping_uidx_to_cid(train_cascades)
list_uids = get_list_infl_users()
list_tweetids = find_tweetid_for_userid(list_uids)
print(list_uids[0:5], list_tweetids[0:5])
ext = np.vstack([list_uids, list_tweetids]).transpose()
np.savetxt('../output/all_kwon/kwon/infl_uids_tweetids.txt', ext, fmt='%d')

[39357, 41645179, 51828921, 1688, 41625921] [3691337196, 1844105043, 3599725603, 3869391368, 1752867728]


# User Features Extraction (Influential Users)

In [52]:
# KWON
# --------------
fake_component = 0
data = "kwon"
data_dir = "../data/{}/".format(data)
cascades_file = data_dir + 'cascades.txt'
labels_file = data_dir + 'labels.txt'
cascade_names = data_dir + 'cascade_names.txt'

userfeat_file = 'kwon_dataset/sub_user_info_share.txt'

output = "all_kwon/kwon"
output_dir = "../output/{}/".format(output)
idx2u_file = output_dir + 'idx2u.txt'
selected_infl_file = output_dir + 'selected_influential_users.tsv'
casnames = np.loadtxt(cascade_names, dtype=np.str)

train_cascades = read_cascades_file(cascades_file)
train_labels = np.loadtxt(labels_file)
print('num cascades', len(train_cascades), len(train_labels))
u_t, u_f, sorted_users = eng_count(train_cascades, train_labels)
fake_ind = np.where(train_labels == 1)[0]
fake_cascades = np.array(train_cascades)[fake_ind]

ufeat = pd.read_csv(userfeat_file, sep='\t', header=None) # followers, followees (friends), posts
ufeat.columns = ['userid', 'followers', 'friends/followees', 'posts']

idx2u = np.loadtxt(idx2u_file)
print('considered users in inference', len(idx2u))

inf_df = pd.read_csv(selected_infl_file, sep='\t', header=0)
print('num selected infl', len(inf_df))
fake_infl = inf_df['selected_{}'.format(fake_component)].apply(lambda x: int(idx2u[x]))
true_infl = inf_df['selected_{}'.format(1-fake_component)].apply(lambda x: int(idx2u[x]))

print(np.array(fake_infl))
print(np.array(true_infl))

# old infl users identified
# fake_infl = [42921853, 45696250, 9428477, 18485804, 37484955, 15538468, 34544545, 80876, 54879, 33361052, 37232745, 33603546, 1680247, 40864273, 134, 20164964, 29144327, 1883240, 47924370, 24916432, 35560818, 15554302, 773, 5272341, 29507615, 29156372, 38769438, 34345266, 2233, 8627533, 14611855, 32269208, 49726906, 28193335, 8098832, 12200968, 55903340, 47588713, 31917878, 19707732, 46248155, 33910097, 55976845, 27047346, 47386549, 9910421, 22678269, 44147043, 35379119, 51841996, 20795579, 37955867, 1002, 46900340, 48467139, 55970765, 17866088, 20853985, 21684, 55972805, 17281580, 18797761, 47872293, 22601310, 7148912, 43816406, 103535, 30331073, 27958549, 17119043, 34686602, 30721844, 28131892, 12068845, 18035830, 27104465, 22978489, 47977103, 46730321, 43369375, 43756699, 12186498, 19444022, 61280, 41050967, 31441663, 42926119, 55991030, 10688944, 16358632, 23219078, 5297007, 2150040, 35332369, 5773920, 68059, 42401663, 27547127, 15491837, 4667953]
# true_infl = [30122132, 9456, 1041957, 55931092, 52656642, 51298452, 2329550, 74679, 9761160, 42598690, 12987474, 1083172, 25480395, 24007123, 53616317, 6934529, 22041886, 25869801, 20510188, 37057701, 42946550, 23803578, 21417341, 31665361, 29264958, 16639123, 28117383, 5396879, 30971958, 23086420, 28776904, 80876, 40591, 49782598, 48911973, 10501473, 9022728, 11494548, 47350415, 55774980, 39515674, 42837, 17072, 49272331, 81419, 42760, 24058200, 10041, 30404702, 6365957, 11749083, 25672074, 14748530, 52263483, 166747, 45543326, 15315324, 20716299, 5903540, 34420488, 40075183, 23246755, 56017762, 43560260, 15191995, 33786128, 360451, 25200040, 16939, 23539100, 16759010, 56023202, 3420957, 9645533, 55935602, 41127254, 7414712, 54917, 52244910, 42976, 29156372, 24773084, 32106161, 35444685, 40176, 46945086, 54780470, 80877, 49755942, 19047950, 9656071, 38410, 11209421, 6481132, 49614, 35775165, 21719470, 22525981, 65561, 825]
# print('fake infl list', list(fake_infl))
# print('true infl list', list(true_infl))


"""
Second: Find features of influential users selected. # followers, # posts, description # eng_t # eng_f
"""

def get_cas_names(uid):
    names_list = []
    for tr_cas, name in zip(fake_cascades, casnames[fake_ind]):
        users = set(tr_cas[:, 0])
        if uid in users:
            names_list.append(name)
        # break
    return names_list
        
def extract_feats(infl, u_t, u_f, strname):
    extracted_df = ufeat[ufeat['userid'].isin(infl)]
    extracted_df['u_t'] = extracted_df['userid'].apply(lambda x: u_t[x] if x in u_t else 0)
    extracted_df['u_f'] = extracted_df['userid'].apply(lambda x: u_f[x] if x in u_f else 0)
    # print(infl, infl[0], type(infl[0]))
    extracted_df['features'] = [dict_ufeats[int(uid)] if int(uid) in dict_ufeats else () for uid in extracted_df['userid']]
    
    extracted_df['casnames'] = [get_cas_names(uid) for uid in extracted_df['userid']]
    extracted_df = extracted_df.set_index('userid').loc[infl]
    # print(extracted_df.head(5))
    # print(extracted_df.head(30))
    # extracted_df.to_csv('../output/all_kwon/kwon/extracted_user_feats_{}.csv'.format(strname))
    

# print('-------> true infl feats\n')
# extract_feats(true_infl, u_t, u_f, 'true_infl')
print('-------> fake infl feats\n')
extract_feats(fake_infl, u_t, u_f, 'fake_infl')
# print('-------> random users \n')
# np.random.seed(145)
# random_users = [idx2u[c] for c in np.random.choice(len(idx2u), len(true_infl), replace=False)]
# extract_feats(random_users, u_t, u_f)


num cascades 111 111
t/f 51 60
u_t, u_f, tot 76041 47869 123910
users large eng > 5 2457
users large eng > 10 871
considered users in inference 2930
num selected infl 100
[   39357 41645179 51828921     1688 41625921   904660 55640892 45357704
   117511 31745148 49130864 46283660 31441663 46161420 54595164 39025741
 16983873    10045 51489852 53382638 19771216    18987    85853 39003758
 26101679 54795103 31749799 40531228 11402038 31665361  2187715 54888609
 55762225 12919420 26206026 15191995 44083204  9866568      892 33019416
  4270645 36020039 47509244    49614 51093837 45924090      599     9581
 34949244 30715157 52174399 34664714    40179 45241472   600250 26869787
 33498358    88703 45887855    46849 34753016 12186498 56001060    81502
 10179069    10054 25980076 37733084 28768064 51631070 29708909 23813323
 46643782 56009279 46060230  2447156 40134854 55922657 17236288 39758253
    73058 14808579 36503735 43569984  2267067 32691858 41792135 34882960
 25900052 47386549 4673040

In [31]:
dict_ufeats[2267067]

('hendricius', 'Software engineer + geek', False)

In [32]:
u_f[2267067]

74